# Car Wash Locator Using Overpass API and Nominatim

In [2]:
!pip install requests folium pandas geopy streamlit streamlit-folium tqdm


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\CHonSa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [48]:
import requests
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from tqdm import tqdm


In [49]:
geolocator = Nominatim(user_agent="car-wash-locator")

def reverse_geocode(lat, lon, max_retries=5):
    for attempt in range(max_retries):
        try:
            location = geolocator.reverse((lat, lon), language='en', timeout=10)
            return location.address if location else None
        except GeocoderTimedOut:
            print(f"⏳ Timeout at ({lat}, {lon}) – retrying {attempt + 1}/{max_retries}")
            time.sleep(2)
        except Exception as e:
            print(f"⚠️ Failed to geocode ({lat}, {lon}): {e}")
            return None
        time.sleep(1.2)
    return None


In [50]:
# Change "Sydney" to any desired city or region
query = """
[out:json][timeout:60];
area["name"="Melbourne"]->.a;
(
  node["amenity"="car_wash"]["boundary"="administrative"]["admin_level"="6"](area.a);
  way["amenity"="car_wash"](area.a);
  relation["amenity"="car_wash"](area.a);
);
out center;
"""

url = "https://overpass-api.de/api/interpreter"
response = requests.post(url, data={'data': query})
data = response.json()


In [51]:
results = []

for i, el in tqdm(enumerate(data['elements']), total=len(data['elements'])):
    tags = el.get('tags', {})
    lat = el.get('lat') or el.get('center', {}).get('lat')
    lon = el.get('lon') or el.get('center', {}).get('lon')

    address = reverse_geocode(lat, lon)

    results.append({
        'name': tags.get('name') or f"Unnamed ({lat:.4f}, {lon:.4f})",
        'lat': lat,
        'lon': lon,
        'address': address,
        'website': tags.get('website'),
        'contact:website': tags.get('contact:website'),
        'contact:phone': tags.get('contact:phone'),
        'opening_hours': tags.get('opening_hours'),
        'operator': tags.get('operator'),
        'brand': tags.get('brand')
    })

    if i % 10 == 0:
        pd.DataFrame(results).to_csv("partial_results.csv", index=False)

df = pd.DataFrame(results)
df.to_csv("final_results.csv", index=False)
df.head()


  0%|          | 0/185 [00:00<?, ?it/s]

100%|██████████| 185/185 [03:05<00:00,  1.00s/it]


,name,lat,lon,address,website,contact:website,contact:phone,opening_hours,operator,brand
0,Superpower Car Wash,-37.846113,145.226601,"Ashley Street, Wantirna, Melbourne, City of Kn...",None,None,None,None,None,None
1,"Unnamed (-38.1127, 145.1560)",-38.112741,145.155967,"Seaford Road, Seaford, Melbourne, City of Fran...",None,None,None,None,None,None
2,"Unnamed (-38.1407, 145.1228)",-38.140694,145.122783,"Shell, Beach Street, Frankston, Melbourne, Cit...",None,None,None,None,None,None
3,"Unnamed (-38.1655, 145.1377)",-38.165455,145.137719,"BP, 4, Golf Links Road, Frankston, Melbourne, ...",None,None,None,None,None,None
4,"Unnamed (-38.1542, 145.1638)",-38.154182,145.163784,"7-Eleven, Cranbourne Road, Frankston, Melbourn...",None,None,None,None,None,None


In [41]:
# Example: Your car wash DataFrame (replace this with your real one)
# df = pd.read_csv("car_wash_data.csv")

# -----------------------------
# Function to search nearby named elements
# -----------------------------
def get_nearby_named_elements(lat, lon, radius=50):
    query = f"""
    [out:json][timeout:25];
    (
      node(around:{radius}, {lat}, {lon})["name"];
      way(around:{radius}, {lat}, {lon})["name"];
      relation(around:{radius}, {lat}, {lon})["name"];
    );
    out center;
    """
    url = "https://overpass-api.de/api/interpreter"
    try:
        response = requests.post(url, data={'data': query}, timeout=60)
        if response.ok:
            return response.json().get("elements", [])
        else:
            print("⚠️ Overpass returned error:", response.status_code)
    except Exception as e:
        print(f"❌ Error with Overpass query: {e}")
    return []

# -----------------------------
# Function to enrich one row
# -----------------------------
def try_enrich_car_wash(row, radius=1000):
    if not isinstance(row.get("name", ""), str) or not row["name"].startswith("Unnamed"):
        return row  # Already has a name, skip

    lat, lon = row["lat"], row["lon"]
    
    time.sleep(1.5)  # Be polite to Overpass API

    elements = get_nearby_named_elements(lat, lon, radius)

    for el in elements:
        tags = el.get("tags", {})
        name_candidate = tags.get("name", "")
        if "car wash" in name_candidate.lower():
            row["name"] = name_candidate
            row["website"] = tags.get("website", row.get("website"))
            row["contact:phone"] = tags.get("contact:phone", row.get("contact:phone"))
            break  # Found good match
    print(f"🔍 Checking unnamed car wash at ({lat}, {lon})... {row['name']}")
    return row

# -----------------------------
# Apply to your DataFrame
# -----------------------------
# Make sure you have 'name', 'lat', 'lon' columns
df_enriched = df.copy()
df_enriched = df_enriched.apply(try_enrich_car_wash, axis=1)

# -----------------------------
# Save or review the result
# -----------------------------
df_enriched.to_csv("enriched_car_washes.csv", index=False)
df_enriched[df_enriched["name"].str.startswith("Unnamed") == False].head()


🔍 Checking unnamed car wash at (-33.8065112, 151.0871429)... Unnamed (-33.8065, 151.0871)
🔍 Checking unnamed car wash at (-33.8924901, 151.0661296)... Green Car Hand Car Wash and Cafe
🔍 Checking unnamed car wash at (-33.7648421, 150.8250198)... IMO Car Wash
🔍 Checking unnamed car wash at (-33.761653, 151.2070983)... Unnamed (-33.7617, 151.2071)
🔍 Checking unnamed car wash at (-34.0487406, 150.7561556)... IMO Car Wash
🔍 Checking unnamed car wash at (-33.7691545, 151.0320622)... Prime Shine Car Wash & Cafe
🔍 Checking unnamed car wash at (-33.7517814, 150.9151904)... Unnamed (-33.7518, 150.9152)
🔍 Checking unnamed car wash at (-33.8700215, 150.9581552)... Unnamed (-33.8700, 150.9582)
🔍 Checking unnamed car wash at (-33.9054709, 151.0415438)... Unnamed (-33.9055, 151.0415)
🔍 Checking unnamed car wash at (-33.7912253, 151.1312304)... Unnamed (-33.7912, 151.1312)
🔍 Checking unnamed car wash at (-33.8621506, 150.9391503)... Unnamed (-33.8622, 150.9392)
🔍 Checking unnamed car wash at (-33.8227

,name,lat,lon,address,website,contact:website,contact:phone,opening_hours,operator,brand
0,7-Eleven,-34.061688,150.809713,"7-Eleven, Blaxland Road, Campbelltown, Sydney,...",None,None,None,None,None,None
1,Jack's Prestige Car Wash & Cafe,-33.812508,151.172259,"Jack's Prestige Car Wash & Cafe, Lane Cove Tun...",None,None,None,None,None,None
3,Maxi Shine,-33.622057,150.836310,"Maxi Shine, Windsor - Parramatta Cycleway, Mul...",None,None,None,None,None,None
4,Green Car Hand Car Wash and Cafe,-33.892490,151.066130,"Angophora Grove, Greenacre, Sydney, Canterbury...",None,None,None,None,None,None
5,IMO Car Wash,-33.766405,150.817486,"Carlisle Avenue, Mount Druitt, Sydney, Blackto...",None,None,None,None,None,IMO Car Wash
